# Import libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import joblib

# Load Dataset and Analyze

In [ ]:
# load the dataset and display the first five rows
df = pd.read_csv("student_data_train.csv")

In [ ]:
# Display summary statistics of the training dataset
print("Summary Statistics of Training Data")
print(df.describe())

In [ ]:
# Check for missing values in the training dataset
print("Checking for missing valuse in Training Data")
df.isnull().sum()

# Data Preprocessing

In [ ]:
# Convert categorical variables to numerical using Label Encoding
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

In [ ]:
# split the data into features and target variable
X = df.drop('Performance', axis=1)
y = df['Performance']

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Creation and Training

In [ ]:
# Create the decision tree classifier
clf = DecisionTreeClassifier()

In [ ]:
# Train the model on the training data
clf.fit(X_train, y_train)

# Model Prediction

In [ ]:
# Make prediction on the test data
y_pred = clf.predict(X_test)

# Model Evaluation

In [ ]:
# Calculate and print metrics 
print("Accuracy::", metrics.accuracy_score(y_test, y_pred))
print("Precision::", metrics.precision_score(y_test, y_pred, average='weighted'))
print("Recall::", metrics.recall_score(y_test, y_pred, average='weighted'))
print("F1-Score::", metrics.f1_score(y_test, y_pred, average='weighted'))

In [ ]:
# View classification report
print(metrics.classification_report(y_test, y_pred))

In [ ]:
# Save the trained model to a file
joblib.dump(clf, 'student_performance_predictor.joblib')

# Save the label encoders
joblib.dump(label_encoders, 'label_encoders.joblib')

print("Model saved successfully.")

# Calculate Feature Importance

In [ ]:
importances = clf.feature_importances_
# Create a DataFrame to display feature importances
feature_importances = pd.DataFrame({'Feature' : X.columns, 'Importance' : importances})
feature_importances = feature_importances.sort_values('Importance', ascending=False)
print(feature_importances)

# Visualizations

In [ ]:
# Visualize the Decision Tree
plt.figure(figsize=(20,10))
plot_tree(clf, filled=True, feature_names=X.columns, class_names=label_encoders['Performance'].classes_)
plt.show()

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6)) 
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues') 
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.jpg', dpi=300)
plt.show() 

In [ ]:
# Plot feature importances
plt.figure(figsize=(12, 6))
plt.plot(feature_importances['Feature'], feature_importances['Importance'], marker='o')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importance')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.grid(True)
plt.savefig('feature_importance.jpg', dpi=300)
plt.show()

In [ ]:
# Plot histograms for numerical features
numerical_features = df.select_dtypes(include=['float64', 'int64'])
numerical_features.hist(figsize=(12, 10))
plt.tight_layout()
plt.savefig('feature_visualizations.jpg', dpi=300)
plt.show()

In [ ]:
# Plot correlation matrix
corr = df.corr()

# Create a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.savefig('correlation_heatmap.jpg', dpi=300)
plt.show()

In [ ]:
# Find correlations with Performance
correlations = df.corr()['Performance'].drop('Performance')
correlations = correlations.sort_values(ascending=False)

# Plot the correlations
plt.figure(figsize=(10, 6))
correlations.plot(kind='bar')
plt.title('Correlation with Performance')
plt.ylabel('Correlation Coefficient')
plt.savefig('feature_correlation.jpg', dpi=300)
plt.show()


# Test the Model

In [ ]:
# Load the test dataset
df_test = pd.read_csv("student_data_test.csv")

# Apply label encoding to categorical variables
for column in df_test.select_dtypes(include=['object']).columns:
    df_test[column] = label_encoders[column].transform(df_test[column])

# Split the test dataset into features and target variable
X_test = df_test.drop('Performance', axis=1)
y_test = df_test['Performance']

# Make predictions on the test dataset
y_pred_test = clf.predict(X_test)

# Evaluate the model
print("Accuracy on Test Data:", metrics.accuracy_score(y_test, y_pred_test))
print("Precision on Test Data:", metrics.precision_score(y_test, y_pred_test, average='weighted'))
print("Recall on Test Data:", metrics.recall_score(y_test, y_pred_test, average='weighted'))
print("F1-Score on Test Data:", metrics.f1_score(y_test, y_pred_test, average='weighted'))
print(metrics.classification_report(y_test, y_pred_test))

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred_test), annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix on Test Data')
plt.savefig('confusion_matrix_test.jpg', dpi=300)
plt.show()
